# SquiblyTwo
Detects WMI SquiblyTwo Attack with possible renamed WMI by looking for imphash

## Rule Content
```
- title: SquiblyTwo
  id: 8d63dadf-b91b-4187-87b6-34a1114577ea
  status: experimental
  description: Detects WMI SquiblyTwo Attack with possible renamed WMI by looking
    for imphash
  references:
  - https://subt0x11.blogspot.ch/2018/04/wmicexe-whitelisting-bypass-hacking.html
  - https://twitter.com/mattifestation/status/986280382042595328
  tags:
  - attack.defense_evasion
  - attack.t1047
  author: Markus Neis / Florian Roth
  falsepositives:
  - Unknown
  level: medium
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image:
      - '*\wmic.exe'
      CommandLine:
      - wmic * *format:\"http*
      - wmic * /format:'http
      - wmic * /format:http*
    selection2:
      Imphash:
      - 1B1A3F43BF37B5BFE60751F2EE2F326E
      - 37777A96245A3C74EB217308F3546F4C
      - 9D87C9D67CE724033C0B40CC4CA1B206
      CommandLine:
      - '* *format:\"http*'
      - '* /format:''http'
      - '* /format:http*'
    condition: 1 of them

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:(*\\wmic.exe) AND process_command_line.keyword:(wmic\ *\ *format\:\\\"http* OR wmic\ *\ \/format\:'http OR wmic\ *\ \/format\:http*)) OR (hash_imphash:("1B1A3F43BF37B5BFE60751F2EE2F326E" OR "37777A96245A3C74EB217308F3546F4C" OR "9D87C9D67CE724033C0B40CC4CA1B206") AND process_command_line.keyword:(*\ *format\:\\\"http* OR *\ \/format\:'http OR *\ \/format\:http*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()